这个code的目的是通过使用一个完整的循环，来将之前的步骤多次循环完成这个结果
现在存在的问题是如何存储写出这个loops，需要确保这个递归能够正确的将每一个步骤的结果传递到递归的result中， 这是关于目前写这个loop的主要思路：此处需要假设已经通过使用初始调用API，来对数据集中的结果进行初步的分类，存在一个初始的分类结果，然后通过这个解析函数，将这个初始的分类结果以字典形式存储（将不同的状态的分类结果存储为键值对（其中键为状态类型，值为分类的物种结果）），然后在通过递归每个键值，来将每个部分的结果都存储在results中，再以不同的键作为递归的输入，然后返回递归的结果，将递归的结果存储在result{}中之后在分别调用

In [1]:
# 第一步先把所有需要的包导进去
import json
from openai import OpenAI
import os
import re
import requests

此处先将需要使用到的文件内容，先全部导入进这个环境中，主要包含（形态学矩阵数据，初步解析的数据），其中初步解析的数据是指通过初次调用API选择的initial character根据其状态所得到的物种分类信息，同时解析存储为字典状态。

In [4]:
# 调用环境中的API_KEY
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
# 导入形态学矩阵数据
with open("D:/桌面/TEST-KG/nexus fix/matrix_knowledge_graph_22.json", "r", encoding="utf-8") as file:
    matrix_data = json.load(file)
# 将输入的JSON文件转为字符串
# matrix_data_str = json.dumps(matrix_data, ensure_ascii=False)

# 将initial character的分类结果转换为字典的格式结果，存储在groups中
initial_classification = {
    "1": ["Equisetum_litorale", "Equisetum_palustre"],
    "2": ["Equisetum_pratense", "Equisetum_hyemale", "Equisetum_moorei", "Equisetum_ramosissimum", "Equisetum_trachyodon", "Equisetum_variegatum"],
    "1 and 2": ["Equisetum_arvense", "Equisetum_telmateia", "Equisetum_sylvaticum", "Equisetum_fluviatile"]
}
# 用于存储所有值的列表
groups = []
# 遍历字典中的每一个键值对，并将值存储在列表中，这是为后续设置分布循环设置基础，对于一个字典内容中的各个键值调用存储到一个变量中然后可以多次循环这几个变量最终存储结果
for state, species_list in initial_classification.items():
    groups.append(species_list)
# 打印存储在列表中的结果
for i, group in enumerate(groups, 1):
    print(f"Group {i} species:", group)
group_1_species = groups[0]
group_2_species = groups[1]
group_3_species = groups[2]
print(f"State 1: {group_1_species}",f"State 2: {group_2_species}",f"State 3: {group_3_species}")

Group 1 species: ['Equisetum_litorale', 'Equisetum_palustre']
Group 2 species: ['Equisetum_pratense', 'Equisetum_hyemale', 'Equisetum_moorei', 'Equisetum_ramosissimum', 'Equisetum_trachyodon', 'Equisetum_variegatum']
Group 3 species: ['Equisetum_arvense', 'Equisetum_telmateia', 'Equisetum_sylvaticum', 'Equisetum_fluviatile']
State 1: ['Equisetum_litorale', 'Equisetum_palustre'] State 2: ['Equisetum_pratense', 'Equisetum_hyemale', 'Equisetum_moorei', 'Equisetum_ramosissimum', 'Equisetum_trachyodon', 'Equisetum_variegatum'] State 3: ['Equisetum_arvense', 'Equisetum_telmateia', 'Equisetum_sylvaticum', 'Equisetum_fluviatile']


这个是目前的构建循环的部分，主要是需要写两个function部分：首先是调用API的部分，可以对所有groups中的键调用API的结果保持一致，即构建一个通用的API调用，关于所有的键值都可以使用；第二是构建这个循环，在这个关于循环遍历的function中，重要的一点是如何调用从循环中调用每个键值中存储的subgroup的matrix信息，这样可以减少API的调用负担，同时在这个遍历的function中可以需要重复调用到API对于分类数据的处理，因此如何将这个input作为输入是重要的，其次我认为另外一个重点就是，如何保存每个调用API产生的响应结果，如何存储，如何修饰解析，我认为依我来看，在这个循环中最好是只需要简单的输出最终的结果即可。（还存在一个需要潜在考虑的问题，即一次可以实现几个taxa的分类结果，因为目前我所使用测试的是通过使用一个不算特别大的数据集，一共只有12种的taxa，在经过初次分类之后，基本上就分为了2,4,6，然而其中当taxa为6时，我感觉结果就容易出现一定程度上的错误，所以或许在这个loop中可以设置一个条件语句，当键中的存储的species数量超过一个阈值时，就使用另一个API的调用，即只进行初步分类，然后下一步在进行这个一步直接分类，相当于是一个动态的分类选择，这个idea中需要考虑的一个部分是对于secondary的结果需要进行解析在调用给API，所以在这个if条件下这个部分的调用还之后还需要跟随一个解析的函数）这个想法挺有意思，当然现在当务之急是构建一个loops

In [ ]:
# 此处需要假设已经通过使用初始调用API，来对数据集中的结果进行初步的分类，存在一个初始的分类结果，然后通过这个解析函数，将这个初始的分类结果以字典形式存储（将不同的状态的分类结果存储为键值对（其中键为状态类型，值为分类的物种结果）），然后在通过递归每个键值，来将每个部分的结果都存储在results中，再以不同的键作为递归的输入，然后返回递归的结果，将递归的结果存储在result{}中之后在分别调用
# 此处我需要构建一个API的调用，关于对subgroup的分组
def classify_group(group_species):
    group_matrix = {species: matrix_data[species] for species in group_species}
    group_matrix_str = json.dumps(group_matrix, ensure_ascii=False)
    # 其中我很好奇这样的结果存储最终是什么样的
    messages3 = [
            {"role": "system",
             "content":"""
                 You are a helpful taxonomist assistant.\n
                 You are skilled at calculating the correct information gain to choose the character that best divides species into even groups based on their states.\n
                 Based on the selected character, classify the species into different groups according to their states.\n
                 For each group with more than two species, continue selecting characters to further classify this group until each group only has one species.\n
                 After multiple classifications, determine the final classification levels and record each classifying character and its state.\n
                 Finally, generate a taxonomic key.
                 """},
            {"role": "system","content":"""
                Generate the nested taxonomic key based on the provided morphological matrix. \n
                The process involves selecting a character to classify the species into groups. Repeat this classification within each subgroup until each group contains only one species.
                Information gain measures how much the uncertainty in the dataset is reduced after using a character for classification. It helps in selecting characters that minimize the entropy of the subset after classification, leading to better classification results.
                Please select the classification character for these group's species based on the morphological matrix and information gain methods.
                In the morphological matrix, 'Missing' and 'Not applicable' are invalid states. If a character has invalid states for the group being classified, it should be ignored.
                States are represented by numbers. For example, '1 and 2' means multiple states should be treated as a single state type and this multi-state characterization should not be confused with the single states within it (the state of '3' and '2 and 3' is different state, when you choose the character to based on the state to distinguish need to careful handle).The initial character should have no more than three state types.
                You need to calculate the information gain for each character and choose the highest information gain result. The higher the information gain result, the greater the contribution of the feature to the classification.
                After selecting the initial classification character and categorizing the species based on its state, repeat the process within each subgroup. For each subgroup, select the character with the highest information gain to further classify the species. Continue this process recursively until each group contains only one species.
                Now I will show you the morphological matrix. Please provide the classification character and the categorization of species based on its state. Then, continue to classify each subgroup recursively, showing the chosen character and categorization for each subgroup. Please present the result in a structured format, with each step clearly labeled.
                please don't show how you analysis and calculate, please show me the final result           
            """},
            {"role": "assistant",
             "content": """
                Understood. I will generate the nested taxonomic key based on the provided morphological matrix. Here is a summary of the steps I will follow:\n
                1. The matrix includes all species and their different states for each character.\n
                2. I will select a character to classify the species into groups and repeat this classification within each subgroup until each group contains only one species.\n
                3. I will use information gain to measure how much the uncertainty in the dataset is reduced after using a feature for classification. This helps in selecting features that minimize the entropy of the subset after classification, leading to better classification results.\n
                4. I will select the classification character for the group's species based on the morphological matrix and information gain methods.\n
                5. In the morphological matrix, 'Missing' and 'Not applicable' are considered invalid states. If a character has invalid states for the group being classified, it will be ignored.\n
                6. States are represented by numbers. For example, '2 and 3' means multiple states should be treated as a single state type, and This multi-state characterization should not be confused with the individual states(like '2', '3') within it (such as '3' and '2 and 3'  is the different state, these are two separate states, when i choose character to based on different state to distinguish the species). The classification character should have no more than three state types.\n
                7. I will use information gain to calculate all character and choose the highest information gain result, The higher the Information Gain result, the greater the contribution of the feature to the classification. \n
                8. The final result will provide only the initial classification character and the categorization of species based on its state. \n
                9. Don't need to show how the process about choose, only need to show the final result as nested structure, and i will store result in #character classify result# block
                Please provide the group morphological matrix data so that I can proceed with the classification.
             """},
            # 在此处导入数据类型的时候，考虑直接通过之前在字典中存储的内容，来选择调用字典中不同的键值，以及再结合最开始的
            {"role": "user", "content": f"Here is the group information need to be classify and include the morphological matrix{group_matrix_str}"}
        ]
    
    response =  client.chat.completions.create(
            model="gpt-4o",
            messages=messages3,
            stop=None,
            temperature=0,
            max_tokens=1000,
            n = 1)
    return response.choices[0].message.content
    # 通过print来检查API调用的结果，print(result2)


在写完这个调用的主函数之后，我需要实现的就是通过使用循环来存储最终的结果

In [ ]:
# 这就是之前所说的一种能够产生更加准确的分类结果，在一定程度上减少API调用计算的复杂，从而帮助其发挥更加准确的作用
# 假设这是关于当species数量过多时用到的另外一种API调用，这个部分的API调用的目的是通过将较多数量的species先进行分组在调用之前的相关函数去进行进一步的分类（大套小）
def classify_first_group(group_species):
    group_matrix = {species: matrix_data[species] for species in group_species}
    group_matrix_str = json.dumps(group_matrix, ensure_ascii=False)
    messages = "Here is the group information need to be classify"
    response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            stop=None,
            temperature=0,
            max_tokens=1000,
            n = 1)
    result = response.choices[0].message.content
    print(f"API response for group {group_species}: {result}")
    return result


# 这是一个新的递归方式，其中添加了一个额外的条件语句，来帮助减少API在处理较大数量时species的数量时出现错误的问题，来提高最终结果的准确率
def recursive_classification(groups, final_classification, classification_results):
    while groups:
        try:
            state, current_group = groups.pop(0)
            print(f"Processing group with state: {state}, species: {current_group}")  # 调试信息

            if len(current_group) == 1:
                final_classification[current_group[0]] = current_group
            else:
                if len(current_group) > 6: # 目前测定出来的结果是好像在6的时候，我感觉之前就有点问题了，但是不好说，因为之前我测试得到这个数据时，存在一个额外的API调用负担，我在给不同的groups的结果进行研究时，我每次都将这个完整的矩阵输入进去了，这意味着API需要loading当前的这个数据集，然后还需要选择出groups[i]中对应species的矩阵信息，所以这可能会占用很多额外的负担，导致API响应的结果中内存被一定程度的占有，从而导致这个时候当species超过6种时，就会出现错误(值得考虑的是之前所出现的错误，并不是说存在什么十分直接的标错或者离谱的错误，而是在进行多特征状态时，错误的处理了‘2’和‘2 and 3’将这种多状态特征看做一块了，所以可能API在单次分析上可能会具有更好的效果，我认为应该在10种左右是其极限)
                    # 使用 classify_first_group 细分大于6个 species 的数据集
                    first_classification_result = classify_first_group(current_group)
                    classification_results[state] = first_classification_result  # 存储初步分类结果
                    
                    # 将细分后的结果添加到 groups 以进行进一步分类
                    for new_state, new_group in first_classification_result.items():
                        if len(new_group) > 6:
                            # 如果新组仍然大于6，则递归调用 classify_first_group 继续细分
                            groups.append((new_state, new_group))
                        else:
                            # 新组小于等于6，直接进行分类
                            groups.append((new_state, new_group))
                else:
                    # 使用 classify_group 对小于等于6个 species 的数据集进行分类
                    classification_result = classify_group(current_group)
                    classification_results[state] = classification_result  # 存储API调用结果
                    
                    # 将新的分组结果加入 groups 以进行进一步分类
                    for new_state, new_group in classification_result.items():
                        groups.append((new_state, new_group))

        except Exception as e:
            print(f"Error processing group with state: {state}, species: {current_group}")  # 错误调试信息
            print(f"Exception: {e}")
            raise e
    return final_classification

In [ ]:
#这个函数是之前构建一个包含解析的递归函数，但是我认为我目前所构建的解析函数还不够通用，如果在这个遍历的过程中加入解析的过程，很有可能在存储的字典中会出现一些错误的结果，在最终的结果中也会出现很多意想不到的复杂的错误，所以现在的主要想法是构建一个结果
# def recursive_classification(groups, final_classification):
    # while groups:
        # current_group = groups.pop(0)
        
        # if len(current_group) == 1:
            # final_classification[current_group[0]] = current_group
        # else:
            # classification_result = classify_group(current_group)
            # subgroups = parse_initial_classification(classification_result)
            # for sub_character_state, sub_species_list in subgroups.items():
                # groups.append(sub_species_list)
    
    # return final_classification

#目前这个function呢，则不包括解析的过程，只是构建了一个遍历的循环，来存储每个循环的结果，同时如果需要加上条件语句，我觉得理论上也应该是在这个loop这个位置加入额外的另一种动态的API的调用，例如使用条件语句中if，else，elif多层次的条件命令，设置不同的条件，如len(current_group) == 1， 5 > len(current_group) > 1， len(current_group) > 5等，我感觉好麻烦呀，因为我还需要考虑动态的prompt的内容，但是这种少的步骤确实是可以减少这种计算的负担，从而保证其准确率的
def recursive_classification(groups, final_classification, classification_results):
    while groups:
        current_group = groups.pop(0)
        
        if len(current_group) == 1:
            final_classification[current_group[0]] = current_group
        else:
            classification_result = classify_group(current_group)
            classification_results[current_group[0]] = classification_result  # 存储API调用结果
            # 不进行解析，直接存储结果   
    return final_classification

需要设置一个空的数据集，来存储这些结果同时需要将其一块进行调用，其中如果在考虑对结果的存储有这两种最终的存储列表：final_classification, classification_results；其中第一个存储列表final是用于为一种情况，即初始character直接单独分出来一种species，但是这个问题或许还需要值得考虑的地方是需要再当species数量大于6的时候可能还需要用到这个地方

In [ ]:
# 检查在groups中存在的内容
print(groups)

#创建一个空列表，用于存储最终的结果，不同的结果类型
final_classification = {}
classification_results = {}

# 调用之前构建的遍历循环函数以及该函数中的API调用函数，来存储结果
final_classification = recursive_classification(groups, final_classification, classification_results)
# 输出存储的分类结果
print(classification_results)

当将最终的一个结果存储在final_classification中时，我们需要考虑使用的就是，对这个结果需要进行解析整合，存储的最终结果为字典形式。那么首先需要考虑对不同的API调用的结果分别提取出关键的内容，然后分别进行解析，然后将和这个结果很好的转换成为一个整体，并进行解析

In [ ]:
# 定义一个函数来提取每个结果中的所需部分
def extract_final_taxonomic_key(final_classification):
    match = re.search(r'Final Taxonomic Key(.*)', final_classification, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        return None
# 提取每个结果中的所需部分
# 这个部分是需要进行写一个小循环的，需要分别调用最终结果中所有的循环，突然感觉我之前的那个思路似乎还存在许多更加复杂的部分，因为我要把那个遍历的结果（当那个len超过6时可能会存在复杂的遍历过程）需要正确的存储下来，并还需要考虑正确的解析
final_taxonomic_key1 = extract_final_taxonomic_key(final_classification[0])
final_taxonomic_key2 = extract_final_taxonomic_key(final_classification[1])
final_taxonomic_key3 = extract_final_taxonomic_key(final_classification[2])

# 打印提取的结果
print("Final Taxonomic Key 1:\n", final_taxonomic_key1)
print("Final Taxonomic Key 2:\n", final_taxonomic_key2)
print("Final Taxonomic Key 3:\n", final_taxonomic_key3)

一个简单的例子，可能最终是如何将解析的结果存储的：

In [5]:
result1_1 = """
1. Character18
   - 1 and 2: Equisetum_palustre
   - 1: Equisetum_litorale
"""
result2_1 = """
1. Character8
   - 1: 
     - Character7
       - 1: Equisetum_ramosissimum
       - 2 and 3: 
         - Character26
           - 1: Equisetum_variegatum
           - 2: Equisetum_trachyodon
       - 3: Equisetum_hyemale
   - 2: 
     - Character9
       - 1: Equisetum_moorei
       - 2: Equisetum_pratense
"""
result3_1 ="""
1. Character2
   - 1: 
     - Character10
       - 1: Equisetum_telmateia
       - 2: Equisetum_arvense
   - 2: 
     - Character20
       - 2 and 3: Equisetum_pratense
       - 3: Equisetum_sylvaticum
   - 3: Equisetum_fluviatile
"""
# 定义一个函数来解析每个结果字符串
def parse_result(result):
    lines = result.strip().split('\n')
    result_dict = {}
    stack = [(0, result_dict)]  # 使用元组来记录缩进级别和当前字典
    for line in lines:
        indent_level = len(line) - len(line.lstrip())
        current_dict = stack[-1][1]
        # 调整堆栈以匹配当前缩进级别
        while stack and stack[-1][0] >= indent_level:
            stack.pop()
        
        if ':' in line:
            key, value = line.split(':', 1)
            key = key.strip()
            value = value.strip()
            
            if value:
                current_dict[key] = value
            else:
                current_dict[key] = {}
                stack.append((indent_level, current_dict[key]))
        else:
            key = line.strip()
            current_dict[key] = {}
            stack.append((indent_level, current_dict[key]))
    
    return result_dict
# 解析每个结果字符串
parsed_result1 = parse_result(result1_1)
parsed_result2 = parse_result(result2_1)
parsed_result3 = parse_result(result3_1)
# 检查解析结果是否正确
parsed_results = [parsed_result1, parsed_result2, parsed_result3]
# 可以通过使用print来检查是否中间步骤存在错误：print("Parsed Results:", parsed_results)

# 检查初始分类结果的键数量
initial_keys = list(initial_classification.keys())
# 可以通过使用print来检查是否中间步骤存在错误：print("Initial Keys:", initial_keys)

# 检查是否数量匹配
if len(parsed_results) != len(initial_keys):
    print("Error: Parsed results and initial keys do not match in length.")
else:
    # 构建主字典
    nested_structure = {}
    for key, parsed_result in zip(initial_keys, parsed_results):
        nested_structure[key] = parsed_result
    # 打印嵌套结构
    import json
    print(json.dumps(nested_structure, indent=2))


{
  "1": {
    "1. Character18": {
      "- 1 and 2": "Equisetum_palustre",
      "- 1": "Equisetum_litorale"
    }
  },
  "2": {
    "1. Character8": {
      "- 1": {
        "- Character7": {
          "- 1": "Equisetum_ramosissimum",
          "- 2 and 3": {
            "- Character26": {
              "- 1": "Equisetum_variegatum",
              "- 2": "Equisetum_trachyodon",
              "- 3": "Equisetum_hyemale"
            }
          },
          "- 2": {
            "- Character9": {
              "- 1": "Equisetum_moorei",
              "- 2": "Equisetum_pratense"
            }
          }
        }
      }
    }
  },
  "1 and 2": {
    "1. Character2": {
      "- 1": {
        "- Character10": {
          "- 1": "Equisetum_telmateia",
          "- 2": {
            "- Character20": {
              "- 2 and 3": "Equisetum_pratense",
              "- 3": "Equisetum_fluviatile"
            }
          }
        }
      }
    }
  }
}
